In [21]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

sys.path.append('../')
from apnet.model import APNet, AttRNNSpeechModel
from apnet.layers import PrototypeLayer, WeightedSum

from dcase_models.model.models import SB_CNN, MLP
from dcase_models.data.features import MelSpectrogram, Openl3
from dcase_models.util.files import load_json

import keras.backend as K

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
path = '../experiments'
models = {'APNet': APNet, 'SB_CNN': SB_CNN,
          'AttRNNSpeechModel': AttRNNSpeechModel, 'MLP': MLP}
datasets = ['UrbanSound8k', 'MedleySolosDb', 'GoogleSpeechCommands']
folds = ['fold1', 'test', 'test']



for model_name, model_class in models.items():
    for j, dataset_name in enumerate(datasets):
        exp_folder = os.path.join(path, dataset_name, model_name)
        fold_folder = os.path.join(exp_folder, folds[j])
        
        params_file = os.path.join(exp_folder, 'config.json')
        params = load_json(params_file)
        
        params_model = params['models'][model_name]
 
        kwargs = {}
        if model_name == 'APNet':
            kwargs = {
                'custom_objects': {
                    'PrototypeLayer': PrototypeLayer,
                    'WeightedSum': WeightedSum
                }
            }
        model_container = APNet(
            model=None, model_path=fold_folder,
            #**params_model['model_arguments'],
            **kwargs
            
        )
        number_of_parameters = model_container.model.count_params()
        if model_name == 'MLP':
            features = Openl3(**params['features']['Openl3'])
            number_of_parameters += features.openl3.count_params()
            
        print(model_name, dataset_name, number_of_parameters/1000000)
        

(50, 32)
APNet UrbanSound8k 1.181461
(40, 64)
APNet MedleySolosDb 4.168081
(105, 20)
APNet GoogleSpeechCommands 1.750816
SB_CNN UrbanSound8k 0.856314
SB_CNN MedleySolosDb 1.839224
SB_CNN GoogleSpeechCommands 0.169811
AttRNNSpeechModel UrbanSound8k 0.224965
AttRNNSpeechModel MedleySolosDb 0.290435
AttRNNSpeechModel GoogleSpeechCommands 0.201214
MLP UrbanSound8k 9.48027
MLP MedleySolosDb 9.480012
MLP GoogleSpeechCommands 9.483495
